### Funciona Lista Para Automatizacion

In [30]:

#___________________________________________________________________________________________________________________________________________________________

#                                                       LIIBRERIAS NECESARIAS

import os
import csv
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse  # Agregado para construir la cadena de conexión
import shutil
import tempfile


#___________________________________________________________________________________________________________________________________________________________

#                                                 RELLENO DE NULOS CARGA Y ACTUALIZACION ARCHIVOS GOOGLE DRIVE

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a Google Drive
def subir_archivo(file_id, upload_file_path):
    media = MediaFileUpload(upload_file_path, resumable=True)
    updated_file = drive_service.files().update(fileId=file_id, media_body=media).execute()
    print('Archivo actualizado en Google Drive:', updated_file.get('name'))

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Lista para almacenar los archivos modificados
archivos_modificados = []

# Obtener lista de archivos en la carpeta de Google Drive
results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

# Iterar sobre los archivos
for file in files:
    file_id = file['id']
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file_id, download_path)

    # Leer el archivo CSV y hacer modificaciones
    with open(download_path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        rows = list(reader)
        for row_index, row in enumerate(rows):
            for col_index, cell in enumerate(row):
                if not cell:
                    rows[row_index][col_index] = "no definido"

    # Guardar los cambios en el archivo temporal sin la extensión _temp.csv
    download_path_temp = os.path.join(temp_dir, os.path.splitext(file_name)[0])
    with open(download_path_temp, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

    archivos_modificados.append((file_id, download_path_temp))

# Subir los archivos modificados a Google Drive
for file_id, download_path_temp in archivos_modificados:
    subir_archivo(file_id, download_path_temp)

# Guardar la marca de tiempo de la última ejecución
# Puedes cambiar esto según cómo desees formatear la marca de tiempo
marca_de_tiempo_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
with open("ultima_ejecucion.txt", "w") as file:
    file.write(marca_de_tiempo_actual)





Descargado 100%.
Archivo actualizado en Google Drive: InvoicePurchases12312016_Clean.csv


In [61]:
# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# IDs de las carpetas en Google Drive
source_folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'  # ID de la carpeta de destino
comparison_folder_id = '1ZMAqOISfz-z5kv_4OgGZT5lI1C9S3Xwb'  # ID de la carpeta de comparación
destination_folder_id = '18gXDMFkbWoGDXfRN6JVcmQdi791cHZ9G'  # ID de la carpeta de origen



# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path, folder_name):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargando archivo '{folder_name}/{os.path.basename(download_path)}' desde la carpeta '{folder_name}'.")

# Función para subir un archivo a una carpeta específica en Google Drive
def subir_archivo_a_carpeta(upload_file_path, folder_id, file_name):
    media = MediaFileUpload(upload_file_path, resumable=True)
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }
    drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'Archivo {file_name} subido a Google Drive en la carpeta especificada: {folder_id}')

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Obtener lista de archivos en la carpeta de origen de Google Drive
results = drive_service.files().list(q=f"'{source_folder_id}' in parents", fields='files(id, name)').execute()
source_files = results.get('files', [])

# Leer los archivos de la carpeta de origen y almacenarlos en un diccionario
origen_data = {}
for file in source_files:
    file_id = file['id']
    file_name = file['name']
    download_path_origen = os.path.join(temp_dir, file_name)
    
    # Descargar archivo de la carpeta de origen
    descargar_archivo(file_id, download_path_origen, 'Carpeta de origen')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    origen_data[file_name] = pd.read_csv(download_path_origen)
    print(f"Datos del archivo de origen '{file_name}':")

# Obtener lista de archivos en la carpeta de comparación de Google Drive
results_comparison = drive_service.files().list(q=f"'{comparison_folder_id}' in parents", fields='files(id, name)').execute()
comparison_files_list = results_comparison.get('files', [])

# Definir comparison_files como un diccionario vacío para almacenar los nombres de archivo y sus datos
comparison_files = {}

# Leer los archivos de la carpeta de comparación y almacenarlos en un diccionario
for file in comparison_files_list:
    file_id = file['id']
    file_name = file['name']
    download_path_comparacion = os.path.join(temp_dir, f"comparison_{file_name}")
    
    # Descargar archivo de la carpeta de comparación
    descargar_archivo(file_id, download_path_comparacion, 'Carpeta de comparación')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    comparison_files[file_name] = pd.read_csv(download_path_comparacion)
    print(f"Datos del archivo de comparación '{file_name}':")
    
# Realizar la comparación entre los datos de origen y los datos de comparación
for file_name, df_comparacion in comparison_files.items():
    if file_name in origen_data:
        print(f"Realizando comparación entre los datos de '{file_name}' en la carpeta de origen y en la carpeta de comparación.")
        
        df_origen = origen_data[file_name]
        
        # Verificar y convertir las columnas para que tengan el mismo tipo de datos
        columnas_comunes = list(set(df_origen.columns) & set(df_comparacion.columns))
        for columna in columnas_comunes:
            tipo_origen = df_origen[columna].dtype
            tipo_comparacion = df_comparacion[columna].dtype
            if tipo_origen != tipo_comparacion:
                if tipo_origen == 'object' or tipo_comparacion == 'object':
                    # Convertir a tipo de datos string si alguna de las columnas es de tipo object
                    df_origen[columna] = df_origen[columna].astype(str)
                    df_comparacion[columna] = df_comparacion[columna].astype(str)
                else:
                    # Convertir ambas columnas al tipo de datos más general (por ejemplo, float64)
                    tipo_general = max(tipo_origen, tipo_comparacion)
                    df_origen[columna] = df_origen[columna].astype(tipo_general)
                    df_comparacion[columna] = df_comparacion[columna].astype(tipo_general)

        # Comparar y obtener las filas nuevas (las filas que están en df_origen pero no en df_comparacion)
        df_nuevas_filas = df_origen.merge(df_comparacion, indicator=True, how='left').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge'])
        if not df_nuevas_filas.empty:
            print(f"Se encontraron filas nuevas en el archivo '{file_name}':")
            

                # Función para subir un archivo a una carpeta específica en Google Drive
            def subir_archivo_a_carpeta(upload_file_path, folder_id, file_name):
                # Borrar todos los archivos existentes en la carpeta de destino
                files = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id)').execute()
                for file in files.get('files', []):
                    drive_service.files().delete(fileId=file.get('id')).execute()
                    print(f"Archivo '{file_name}' eliminado de la carpeta de destino.")

                # Subir el archivo especificado a la carpeta de destino
                media = MediaFileUpload(upload_file_path, resumable=True)
                file_metadata = {
                    'name': file_name,
                    'parents': [folder_id]
                }
                drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
                print(f'Archivo {file_name} subido a Google Drive en la carpeta especificada: {folder_id}')

            # Subir el DataFrame con las filas nuevas a la carpeta de destino
            nuevo_nombre_path = os.path.join(temp_dir, file_name)
            with open(nuevo_nombre_path, 'w', newline='') as file:
                df_nuevas_filas.to_csv(file, index=False)
            subir_archivo_a_carpeta(nuevo_nombre_path, destination_folder_id, file_name)
            print(f"El archivo '{file_name}' con las filas nuevas se ha subido correctamente a la carpeta de destino.")
        else:
            print(f"No se encontraron filas nuevas en el archivo '{file_name}'.")
    else:
        print(f"No se encontró el archivo '{file_name}' en la carpeta de origen.")


Descargando archivo 'Carpeta de origen/InvoicePurchases12312016_Clean.csv' desde la carpeta 'Carpeta de origen'.
Datos del archivo de origen 'InvoicePurchases12312016_Clean.csv':
Descargando archivo 'Carpeta de origen/2017PurchasePricesDec_Clean.csv' desde la carpeta 'Carpeta de origen'.
Datos del archivo de origen '2017PurchasePricesDec_Clean.csv':
Descargando archivo 'Carpeta de comparación/comparison_2017PurchasePricesDec_Clean.csv' desde la carpeta 'Carpeta de comparación'.
Datos del archivo de comparación '2017PurchasePricesDec_Clean.csv':
Descargando archivo 'Carpeta de comparación/comparison_InvoicePurchases12312016_Clean.csv' desde la carpeta 'Carpeta de comparación'.
Datos del archivo de comparación 'InvoicePurchases12312016_Clean.csv':
Realizando comparación entre los datos de '2017PurchasePricesDec_Clean.csv' en la carpeta de origen y en la carpeta de comparación.
Se encontraron filas nuevas en el archivo '2017PurchasePricesDec_Clean.csv':
Archivo '2017PurchasePricesDec_Clea

In [42]:

#___________________________________________________________________________________________________________________________________________________________

#                                               CARGA DE DATOS SQL

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '18gXDMFkbWoGDXfRN6JVcmQdi791cHZ9G'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = '/ruta/a/tu/directorio/base'
csv_folder_name = 'archivos_csv_google_drive'
local_directory = os.path.join(base_directory, csv_folder_name)

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        download_path = os.path.join(local_directory, file_name)
        request = drive_service.files().get_media(fileId=file_id)
        with open(download_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

    # Cambiar el delimitador de todos los archivos CSV en el directorio
    # Obtener la lista de archivos CSV en el directorio
    csv_files = os.listdir(local_directory)

    # Iterar sobre cada archivo CSV y cambiar el delimitador
    for file_name in csv_files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(local_directory, file_name)
            # Leer el archivo CSV con la coma como delimitador original
            df = pd.read_csv(file_path)
            # Cambiar el delimitador de la coma a punto y coma
            df.to_csv(file_path, sep=';', index=False)

    print("Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.")


# Función para cargar datos desde archivos CSV a una base de datos SQL Server
def load_csv_data_to_sql(csv_directory, conn_str):
    files = os.listdir(csv_directory)
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(csv_directory, file)
            table_name = os.path.splitext(file)[0]  # Utilizar el nombre del archivo CSV como nombre de la tabla
            
            # Define el delimitador predeterminado
            delimiter = ';'

                
            # Define una función de redondeo personalizada
            def custom_round(value):
                if isinstance(value, float):
                    return round(value, 2)
                return value
            
            # Leer el archivo CSV con el delimitador correcto y aplicar la función de redondeo personalizada
            df = pd.read_csv(file_path, delimiter=delimiter, converters={i: custom_round for i in range(10)})  # ajusta el rango 10
            
            # Crear el motor de conexión a la base de datos
            engine = create_engine(conn_str)
            
            # Cargar los datos en SQL Server
            df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            print(f"Data from {file} loaded into SQL table {table_name}")

# Ejemplo de uso
if __name__ == "__main__":
    # Crear el directorio local si no existe
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)

    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)
    
    # Construir la cadena de conexión para SQLAlchemy
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=JULIAN;DATABASE=Top-Drinks;Trusted_Connection=yes;")
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    
    # Cargar datos desde archivos CSV a una base de datos SQL Server
    load_csv_data_to_sql(local_directory, conn_str)


Removed old CSV file: InvoicePurchases12312016_Clean.csv
Removed old CSV file: new_InvoicePurchases12312016_Clean.csv
Downloaded 100% of InvoicePurchases12312016_Clean.csv
Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.
Data from InvoicePurchases12312016_Clean.csv loaded into SQL table InvoicePurchases12312016_Clean


In [59]:
#___________________________________________________________________________________________________________________________________________________________

#                                                       LIBRERIAS NECESARIAS

import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import tempfile

#___________________________________________________________________________________________________________________________________________________________

#                                                 DESCARGA Y CARGA DE ARCHIVOS ENTRE CARPETAS DE GOOGLE DRIVE

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta de origen en Google Drive
source_folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'
# ID de la carpeta de destino en Google Drive
destination_id = '1ZMAqOISfz-z5kv_4OgGZT5lI1C9S3Xwb'  # Reemplaza 'your_destination_folder_id' con el ID de tu carpeta de destino

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Función para subir un archivo a una carpeta específica en Google Drive
def subir_archivo_a_carpeta(upload_file_path, file_name):
    # Buscar el archivo en la carpeta de destino por su nombre
    response = drive_service.files().list(
        q=f"name='{file_name}' and '{destination_id}' in parents",
        fields='files(id)'
    ).execute()
    
    # Si se encuentra el archivo, obtener su ID y actualizarlo
    if response.get('files', []):
        file_id = response['files'][0]['id']
        media = MediaFileUpload(upload_file_path)
        drive_service.files().update(fileId=file_id, media_body=media).execute()
        print(f'Archivo actualizado en Google Drive: {file_name}')
    else:
        # Si el archivo no existe, crear uno nuevo en la carpeta de destino
        media = MediaFileUpload(upload_file_path, resumable=True)
        file_metadata = {
            'name': file_name,
            'parents': [destination_id]
        }
        drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print(f'Archivo subido a Google Drive en la carpeta especificada: {destination_id}')

# Obtener lista de archivos en la carpeta de origen de Google Drive
results = drive_service.files().list(q=f"'{source_folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

# Iterar sobre los archivos, descargarlos y luego cargarlos en la carpeta de destino
for file in files:
    file_name = file['name']
    download_path = os.path.join(temp_dir, file_name)

    # Descargar el archivo de Google Drive
    descargar_archivo(file['id'], download_path)

    # Subir el archivo a la carpeta de destino
    subir_archivo_a_carpeta(download_path, file_name)

print("Archivos descargados y sobrescritos en Google Drive.")




Descargado 100%.
Archivo actualizado en Google Drive: InvoicePurchases12312016_Clean.csv
Archivos descargados y sobrescritos en Google Drive.


### pruebas


##### funciona pero sube todo el archivo


In [31]:
#___________________________________________________________________________________________________________________________________________________________

#                                                       LIBRERIAS NECESARIAS

import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
import pandas as pd
import tempfile

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# IDs de las carpetas en Google Drive
source_folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'  # ID de la carpeta de destino
comparison_folder_id = '1ZMAqOISfz-z5kv_4OgGZT5lI1C9S3Xwb'  # ID de la carpeta de comparación
destination_folder_id  = '18gXDMFkbWoGDXfRN6JVcmQdi791cHZ9G'  # ID de la carpeta de origen

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path, folder_name):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargando archivo '{folder_name}/{os.path.basename(download_path)}' desde la carpeta '{folder_name}'.")

# Función para subir un archivo a una carpeta específica en Google Drive
def subir_archivo_a_carpeta(upload_file_path, folder_id, file_name):
    media = MediaFileUpload(upload_file_path, resumable=True)
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }
    drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'Archivo {file_name} subido a Google Drive en la carpeta especificada: {folder_id}')

# Directorio temporal para manipular archivos
temp_dir = tempfile.gettempdir()

# Obtener lista de archivos en la carpeta de origen de Google Drive
results = drive_service.files().list(q=f"'{source_folder_id}' in parents", fields='files(id, name)').execute()
source_files = results.get('files', [])

# Leer los archivos de la carpeta de origen y almacenarlos en un diccionario
origen_data = {}
for file in source_files:
    file_id = file['id']
    file_name = file['name']
    download_path_origen = os.path.join(temp_dir, file_name)
    
    # Descargar archivo de la carpeta de origen
    descargar_archivo(file_id, download_path_origen, 'Carpeta de origen')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    origen_data[file_name] = pd.read_csv(download_path_origen)
    print(f"Datos del archivo de origen '{file_name}':")
    print(origen_data[file_name].head())

# Obtener lista de archivos en la carpeta de comparación de Google Drive
results_comparison = drive_service.files().list(q=f"'{comparison_folder_id}' in parents", fields='files(id, name)').execute()
comparison_files_list = results_comparison.get('files', [])

# Definir comparison_files como un diccionario vacío para almacenar los nombres de archivo y sus datos
comparison_files = {}

# Leer los archivos de la carpeta de comparación y almacenarlos en un diccionario
for file in comparison_files_list:
    file_id = file['id']
    file_name = file['name']
    download_path_comparacion = os.path.join(temp_dir, f"comparison_{file_name}")
    
    # Descargar archivo de la carpeta de comparación
    descargar_archivo(file_id, download_path_comparacion, 'Carpeta de comparación')
    
    # Leer el archivo CSV y almacenar los datos en el diccionario
    comparison_files[file_name] = pd.read_csv(download_path_comparacion)
    print(f"Datos del archivo de comparación '{file_name}':")
    print(comparison_files[file_name].head())

# Realizar la comparación entre los datos de origen y los datos de comparación
for file_name, df_comparacion in comparison_files.items():
    if file_name in origen_data:
        print(f"Realizando comparación entre los datos de '{file_name}' en la carpeta de origen y en la carpeta de comparación.")
        
        df_origen = origen_data[file_name]
        
        # Comparar y obtener las filas nuevas
        df_nuevas_filas = df_origen[~df_origen.apply(tuple, axis=1).isin(df_comparacion.apply(tuple, axis=1))]

        if not df_nuevas_filas.empty:
            print(f"Se encontraron filas nuevas en el archivo '{file_name}':")
            print(df_nuevas_filas)
            # Subir el DataFrame con las filas nuevas a la carpeta de destino
            nuevo_nombre_archivo = f"new_{file_name}"
            nuevo_nombre_path = os.path.join(temp_dir, nuevo_nombre_archivo)
            df_nuevas_filas.to_csv(nuevo_nombre_path, index=False)
            subir_archivo_a_carpeta(nuevo_nombre_path, destination_folder_id, nuevo_nombre_archivo)
            print(f"El archivo {nuevo_nombre_archivo} con las filas nuevas se ha subido correctamente a la carpeta de destino.")
        else:
            print(f"No se encontraron filas nuevas en el archivo '{file_name}'.")
    else:
        print(f"No se encontró el archivo {file_name} en la carpeta de origen.")


Descargando archivo 'Carpeta de origen/InvoicePurchases12312016_Clean.csv' desde la carpeta 'Carpeta de origen'.
Datos del archivo de origen 'InvoicePurchases12312016_Clean.csv':
   VendorNumber                   VendorName InvoiceDate PONumber      PODate  \
0           105  ALTAMAR BRANDS LLC           2016-01-04     8124  2015-12-21   
1          4466  AMERICAN VINTAGE BEVERAGE    2016-01-07     8137  2015-12-22   
2           388  ATLANTIC IMPORTING COMPANY   2016-01-09     8169  2015-12-24   
3           480  BACARDI USA INC              2016-01-12     8106  2015-12-20   
4           516  BANFI PRODUCTS CORP          2016-01-07     8170  2015-12-24   

      PayDate Quantity    Dollars  Freight  
0  2016-02-16        6     214.26     3.47  
1  2016-02-21       15     140.55     8.57  
2  2016-02-16        5      106.6     4.61  
3  2016-02-05    10100  137483.78  2935.20  
4  2016-02-12     1935   15527.25   429.20  
Descargando archivo 'Carpeta de origen/PurchasesFINAL12312016_Cl

KeyboardInterrupt: 

#### cambio de datos nulos y actualizacion del archivo

In [ ]:
import pyodbc
# Función para listar las tablas en la base de datos
def listar_tablas():
    try:


# Define the connection string
        conn_str = (
            'DRIVER={ODBC Driver 17 for SQL Server};'
            'SERVER=JULIAN;'
            'DATABASE=Top-Drinks;'
            'Trusted_Connection=yes;'
        )

        # Establish a connection
        try:
            conn = pyodbc.connect(conn_str)
            print("Conexión exitosa")
            # Create a cursor
            cursor = conn.cursor()

            # Ahora puedes ejecutar comandos SQL usando el cursor
            # cursor.execute("SELECT * FROM tu_tabla")
            # rows = cursor.fetchall()
            # for row in rows:
            #     print(row)
        except pyodbc.Error as e:
            print("Error en la conexión a SQL Server:", e)


        # Now you can execute SQL commands using the cursor

        # Obtener la lista de tablas
        cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
        tablas = cursor.fetchall()

        print("Tablas en la base de datos:")
        for tabla in tablas:
            print(tabla[0])

        cursor.close()
        conn.close()
    except Exception as e:
        print(f'Error en la conexión a SQL Server: {str(e)}')

# Llamar a la función para listar las tablas
listar_tablas()

    


Conexión exitosa
Tablas en la base de datos:
SalesFINAL12312016_Clean
PurchasesFINAL12312016_Clean
2017PurchasePricesDec_Clean
BegInvFINAL12312016_Clean
EndInvFINAL12312016_Clean
InvoicePurchases12312016_Clean


In [ ]:
import os
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = '/ruta/a/tu/directorio/base'
csv_folder_name = 'archivos_csv_google_drive'
local_directory = os.path.join(base_directory, csv_folder_name)

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        download_path = os.path.join(local_directory, file_name)
        request = drive_service.files().get_media(fileId=file_id)
        with open(download_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

    # Cambiar el delimitador de todos los archivos CSV en el directorio
    # Obtener la lista de archivos CSV en el directorio
    csv_files = os.listdir(local_directory)

    # Iterar sobre cada archivo CSV y cambiar el delimitador
    for file_name in csv_files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(local_directory, file_name)
            # Leer el archivo CSV con la coma como delimitador original
            df = pd.read_csv(file_path)
            # Cambiar el delimitador de la coma a punto y coma
            df.to_csv(file_path, sep=';', index=False)

    print("Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.")

    # Verificar la existencia del archivo old_file.csv
    old_file_path = os.path.join(local_directory, "old_file.csv")
    if os.path.exists(old_file_path):
        print(f"El archivo {old_file_path} existe.")
        new_file_path = os.path.join(local_directory, "new_file.csv")
        compare_and_create_new_file(old_file_path, new_file_path)
    else:
        print(f"El archivo {old_file_path} no existe o la ruta es incorrecta.")

# Función para comparar archivos CSV antiguos y nuevos y crear un nuevo archivo con las filas o columnas nuevas
def compare_and_create_new_file(old_file_path, new_file_path):
    old_df = pd.read_csv(old_file_path)
    new_df = pd.read_csv(new_file_path)
    
    # Comparar los DataFrames y encontrar las filas nuevas en el archivo nuevo
    new_rows = new_df[~new_df.isin(old_df)].dropna()
    
    # Guardar las filas nuevas en un nuevo archivo CSV
    new_file_name = os.path.basename(old_file_path)  # Utilizar el nombre del archivo viejo
    new_file_path = os.path.join(local_directory, new_file_name)
    new_rows.to_csv(new_file_path, index=False)
    print(f"Se han creado el archivo {new_file_path} con las filas nuevas.")

# Ejemplo de uso
if __name__ == "__main__":
    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)


Removed old CSV file: 2017PurchasePricesDec_Clean.csv
Removed old CSV file: BegInvFINAL12312016_Clean.csv
Removed old CSV file: EndInvFINAL12312016_Clean.csv
Removed old CSV file: InvoicePurchases12312016_Clean.csv
Removed old CSV file: PurchasesFINAL12312016_Clean.csv
Removed old CSV file: SalesFINAL12312016_Clean.csv
Downloaded 28% of PurchasesFINAL12312016_Clean.csv
Downloaded 57% of PurchasesFINAL12312016_Clean.csv
Downloaded 86% of PurchasesFINAL12312016_Clean.csv
Downloaded 100% of PurchasesFINAL12312016_Clean.csv
Downloaded 82% of SalesFINAL12312016_Clean.csv
Downloaded 100% of SalesFINAL12312016_Clean.csv
Downloaded 100% of BegInvFINAL12312016_Clean.csv
Downloaded 100% of EndInvFINAL12312016_Clean.csv
Downloaded 100% of InvoicePurchases12312016_Clean.csv
Downloaded 100% of 2017PurchasePricesDec_Clean.csv
Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.
El archivo /ruta/a/tu/directorio/base\archivos_csv_google_drive\old_file.csv no existe o l

In [ ]:
import os
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = '/ruta/a/tu/directorio/base'
csv_folder_name = 'archivos_csv_google_drive'
local_directory = os.path.join(base_directory, csv_folder_name)

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    print("Descargando archivos CSV desde Google Drive...")
    
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        download_path = os.path.join(local_directory, file_name)
        request = drive_service.files().get_media(fileId=file_id)
        with open(download_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

    # Cambiar el delimitador de todos los archivos CSV en el directorio
    print("Cambiando delimitadores de archivos CSV...")
    for file_name in os.listdir(local_directory):
        if file_name.endswith('.csv'):
            file_path = os.path.join(local_directory, file_name)
            # Leer el archivo CSV con la coma como delimitador original
            df = pd.read_csv(file_path)
            # Cambiar el delimitador de la coma a punto y coma
            df.to_csv(file_path, sep=';', index=False)

    print("Se han cambiado los delimitadores de todos los archivos CSV en el directorio por ';'.")

# Función para cargar datos desde archivos CSV a una base de datos SQL Server
def load_csv_data_to_sql(csv_directory, conn_str):
    print("Cargando datos desde archivos CSV a SQL Server...")
    for file in os.listdir(csv_directory):
        if file.endswith('.csv'):
            file_path = os.path.join(csv_directory, file)
            table_name = os.path.splitext(file)[0]  # Utilizar el nombre del archivo CSV como nombre de la tabla
            
            # Define el delimitador predeterminado
            delimiter = ';'

            # Define una función de redondeo personalizada
            def custom_round(value):
                if isinstance(value, float):
                    return round(value, 2)
                return value
            
            # Leer el archivo CSV con el delimitador correcto y aplicar la función de redondeo personalizada
            df = pd.read_csv(file_path, delimiter=delimiter, converters={i: custom_round for i in range(10)})  # ajusta el rango 10
            
            # Crear el motor de conexión a la base de datos
            engine = create_engine(conn_str)
            
            # Cargar los datos en SQL Server
            df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            print(f"Data from {file} loaded into SQL table {table_name}")

# Ejemplo de uso
if __name__ == "__main__":
    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)
    
    # Construir la cadena de conexión para SQLAlchemy
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=JULIAN;DATABASE=Top-Drinks;Trusted_Connection=yes )
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    
    # Cargar datos desde archivos CSV a una base de datos SQL Server
    load_csv_data_to_sql(local_directory, conn_str)


SyntaxError: unterminated string literal (detected at line 94) (170457041.py, line 94)

#### no funciono
##### cambiar el tipo de datos desde python


In [ ]:
import os
import urllib
import pandas as pd
from sqlalchemy import create_engine
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import pyodbc
from datetime import datetime, timedelta

# Definir la ruta al archivo JSON de credenciales de Google Drive
credentials_file = 'data-424019-28bfddebf741.json'

# Cargar las credenciales desde el archivo JSON
credentials = service_account.Credentials.from_service_account_file(credentials_file)

# Crear un cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive donde se encuentran los archivos CSV
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Directorio base donde se guardarán los archivos CSV descargados
base_directory = 'C:/Users/julia/Documents'  # Cambiar por la ruta correcta
csv_folder_name = 'archivos_csv_google_drive'

# Función para establecer y validar la ruta base
def establecer_ruta_base(base_path, folder_name):
    local_directory = os.path.join(base_path, folder_name)
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)
    return local_directory

# Variable global para almacenar la hora de la última descarga
last_download_time = None

# Función para descargar archivos CSV de Google Drive
def download_csv_files_from_drive(folder_id, local_directory):
    global last_download_time
    
    # Eliminar archivos CSV antiguos en el directorio local
    for file in os.listdir(local_directory):
        if file.endswith('.csv'):
            os.remove(os.path.join(local_directory, file))
            print(f"Removed old CSV file: {file}")
    
    # Verificar si se ha descargado en la última hora
    if last_download_time is not None and datetime.now() - last_download_time < timedelta(hours=1):
        print("CSV files were downloaded less than an hour ago. Skipping download.")
        return
    
    # Obtener la hora actual menos 1 hora
    one_hour_ago = datetime.now() - timedelta(hours=1)
    
    # Descargar los nuevos archivos CSV desde Google Drive
    results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name, modifiedTime)').execute()
    files = results.get('files', [])
    for file in files:
        file_id = file['id']
        file_name = file['name']
        modified_time = datetime.fromisoformat(file['modifiedTime'][:-1])  # Eliminar la 'Z' del final del timestamp
        # Descargar solo si el archivo ha sido modificado en la última hora
        if modified_time > one_hour_ago:
            download_path = os.path.join(local_directory, file_name)
            request = drive_service.files().get_media(fileId=file_id)
            with open(download_path, 'wb') as fh:
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Downloaded {int(status.progress() * 100)}% of {file_name}")

        
    # Actualizar la hora de la última descarga
    last_download_time = datetime.now()

    # Llamar a la función para imprimir los tipos de datos de cada columna en cada tabla
    print_column_types(local_directory)



# Función para cargar datos desde archivos CSV a una base de datos SQL Server
def load_csv_data_to_sql(csv_directory, conn_str):
    files = os.listdir(csv_directory)
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(csv_directory, file)
            table_name = os.path.splitext(file)[0]  # Utilizar el nombre del archivo CSV como nombre de la tabla
            
            # Verificar si existe un mapeo de tipos de datos para esta tabla
            if table_name in column_types:
                # Obtener el mapeo de tipos de datos para esta tabla
                columns = column_types[table_name]
                
                # Leer el archivo CSV con el delimitador correcto
                delimiter = ';'  # Delimitador predeterminado
                if table_name in ['BegInvFINAL12312016_Clean', 'EndInvFINAL12312016_Clean']:
                    delimiter = ','  # Cambiar el delimitador para archivos específicos
                df = pd.read_csv(file_path, delimiter=delimiter)
                
                # Aplicar conversiones de tipo de datos según el mapeo
                for column, dtype in columns.items():
                    if column in df.columns:
                        df[column] = df[column].astype(dtype)
                
                # Crear el motor de conexión a la base de datos
                engine = create_engine(conn_str)
                
                # Cargar los datos en SQL Server
                df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
                print(f"Data from {file} loaded into SQL table {table_name}")
            else:
                print(f"No column type mapping found for table {table_name}")


                
def print_column_types(local_directory):
    for table_name, columns in column_types.items():
        print(f"Tabla: {table_name}")
        file_path = os.path.join(local_directory, f"{table_name}.csv")
        
        # Leer el archivo CSV
        df = pd.read_csv(file_path)
        
        # Aplicar los tipos de datos especificados en column_types al DataFrame
        for col, dtype in columns.items():
            if col in df.columns:
                if dtype == 'datetime':
                    try:
                        df[col] = pd.to_datetime(df[col])
                    except ValueError:
                        print(f"Error al convertir la columna {col} a tipo de dato datetime en la tabla {table_name}")
                elif dtype == 'nvarchar(max)':
                    df[col] = df[col].astype('object')  # Convertir a tipo 'object'
                else:
                    # Convertir valores no numéricos o desconocidos a NaN antes de la conversión
                    df[col] = pd.to_numeric(df[col], errors='coerce')
                    # Luego, convertir a tipo de dato especificado
                    df[col] = df[col].astype(dtype)
        
        # Imprimir los tipos de datos actualizados
        print(df.dtypes)
        print()




# Crear un diccionario con los nombres de las columnas y los tipos de datos asociados
column_types = {
    'SalesFINAL12312016_Clean': {
        'InventoryId': 'nvarchar(max)',
        'Store': 'int',
        'Brand': 'int',
        'Description': 'nvarchar(max)',
        'Size': 'nvarchar(max)',
        'SalesQuantity': 'int',
        'SalesDollars': 'float',
        'SalesPrice': 'float',
        'SalesDate': 'datetime',
        'Volume': 'int',
        'Classification': 'int',
        'ExciseTax': 'float',
        'VendorNo': 'int',
        'VendorName': 'nvarchar(max)'
    },
    'BegInvFINAL12312016_Clean': {
        'InventoryId': 'nvarchar(max)',
        'Store': 'int',
        'City': 'nvarchar(max)',
        'Brand': 'int',
        'Description': 'nvarchar(max)',
        'Size': 'nvarchar(max)',
        'onHand': 'int',
        'Price': 'float',
        'startDate': 'datetime'
        },
    'InvoicePurchases12312016_Clean': {
        'VendorNumber': 'int',
        'VendorName': 'nvarchar(max)',
        'InvoiceDate': 'datetime',
        'PONumber': 'int',
        'PODate': 'datetime',
        'PayDate': 'datetime',
        'Quantity': 'int',
        'Dollars': 'float',
        'Freight': 'float'
    },
    '2017PurchasePricesDec_Clean': {
        'Brand': 'int',
        'Description': 'nvarchar(max)',
        'Price': 'float',
        'Size': 'nvarchar(max)',
        'Volume': 'int',
        'Classification': 'int',
        'PurchasePrice': 'float',
        'VendorNumber': 'int',
        'VendorName': 'nvarchar(max)'
    },
    'PurchasesFINAL12312016_Clean': {
        'InventoryId': 'nvarchar(max)',
        'Store': 'int',
        'Brand': 'int',
        'Description': 'nvarchar(max)',
        'Size': 'nvarchar(max)',
        'VendorNumber': 'int',
        'VendorName': 'nvarchar(max)',
        'PONumber': 'int',
        'PODate': 'datetime',
        'ReceivingDate': 'datetime',
        'InvoiceDate': 'datetime',
        'PayDate': 'datetime',
        'PurchasePrice': 'float',
        'Quantity': 'int',
        'Dollars': 'float',
        'Classification': 'int'
    },
    'EndInvFINAL12312016_Clean': {
        'InventoryId': 'nvarchar(max)',
        'Store': 'int',
        'City': 'nvarchar(max)',
        'Brand': 'int',
        'Description': 'nvarchar(max)',
        'Size': 'nvarchar(max)',
        'onHand': 'int',
        'Price': 'float',
        'endDate': 'datetime'
    }
}



if __name__ == "__main__":
    # Establecer y validar la ruta base
    local_directory = establecer_ruta_base(base_directory, csv_folder_name)
    
    # Descargar archivos CSV de Google Drive a un directorio local
    download_csv_files_from_drive(folder_id, local_directory)
    
    # Construir la cadena de conexión para SQLAlchemy
    params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};SERVER=JULIAN;DATABASE=Top-Drinks;Trusted_Connection=yes;")
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    
    # Cargar datos desde archivos CSV a una base de datos SQL Server
    load_csv_data_to_sql(local_directory, conn_str)
    
    # Llamar a la función para listar las tablas en la base de datos SQL Server
    listar_tablas()



Removed old CSV file: 2017PurchasePricesDec_Clean.csv
Removed old CSV file: BegInvFINAL12312016_Clean.csv
Removed old CSV file: EndInvFINAL12312016_Clean.csv
Removed old CSV file: InvoicePurchases12312016_Clean.csv
Removed old CSV file: PurchasesFINAL12312016_Clean.csv
Removed old CSV file: SalesFINAL12312016_Clean.csv
Tabla: SalesFINAL12312016_Clean


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/julia/Documents\\archivos_csv_google_drive\\SalesFINAL12312016_Clean.csv'

In [ ]:
# Leer el archivo CSV con las columnas de fecha analizadas correctamente
df = pd.read_csv(file_path, dayfirst=True)
print("Columnas presentes en el archivo CSV:")
print(df.columns)
df = df.astype(columns)
print(df.dtypes)
print()

NameError: name 'file_path' is not defined

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload
import pandas as pd
import os

# Directorio donde se encuentra el archivo JSON de credenciales
directorio_credenciales = "data-424019-28bfddebf741.json"

# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID del archivo a descargar
file_id = 'your-file-id'
download_path = 'downloaded_file.xlsx'  # Puedes cambiar la extensión según el tipo de archivo

# Función para descargar un archivo de Google Drive
def descargar_archivo(file_id, download_path):
    request = drive_service.files().get_media(fileId=file_id)
    with open(download_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Descargado {int(status.progress() * 100)}%.")

# Descargar el archivo
descargar_archivo(file_id, download_path)

# Determinar el tipo de archivo y leerlo con pandas
if download_path.endswith('.csv'):
    df = pd.read_csv(download_path)
elif download_path.endswith('.xlsx'):
    df = pd.read_excel(download_path)
else:
    raise ValueError("Formato de archivo no soportado")

# Mostrar tipos de datos de cada columna
print(df.dtypes)
